# Infosys Quarterly Report Analysis comparison
In this demo, we explore answering complex queries by decomposing them into simpler sub-queries.

In [2]:
!pip install getpass4 pypdf llama-index openai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.5/276.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.1/868.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.8 MB/s eta 0:00:00
  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1846 sha256=c32a6e6023c019b2c8b02ab7c23b15500609eea03495ecb82ffdb6454fe625c8
  Stored in directory: /root/.cache/pip/wheels/a9/16/29/0b45762bf14ad4ba5495cd4ce6

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass("OpenAI Key:")

OpenAI Key:··········


In [ ]:
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.response.pprint_utils import pprint_response
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [ ]:
import faiss
from llama_index.vector_stores import FaissVectorStore

## Configure LLM service

In [ ]:
llm = OpenAI(temperature=0,
             model="text-davinci-003",
             max_tokens=500)

service_context = ServiceContext.from_defaults(llm=llm)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Multiple Documents

In [ ]:
q1_2022_new = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/LLM/Draftcourse/notebooks/ifrs-inr-press-release_q1_2022.pdf"]
).load_data()

q1_2023_new = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/LLM/Draftcourse/notebooks/ifrs-inr-press-release_q1_2023.pdf"]
).load_data()

q1_2024_new = SimpleDirectoryReader(
    input_files=["/content/drive/MyDrive/LLM/Draftcourse/notebooks/ifrs-inr-press-release_q1_2024.pdf"]
).load_data()

In [ ]:
# dimensions of text-ada-embedding-002
d = 1536
faiss_index_m1 = faiss.IndexFlatL2(d)

vector_store_m1 = FaissVectorStore(faiss_index=faiss_index_m1)
storage_context_m1 = StorageContext.from_defaults(vector_store=vector_store_m1)
q1_2022_index_m = VectorStoreIndex.from_documents(q1_2022_new, storage_context=storage_context_m1)


In [ ]:
# dimensions of text-ada-embedding-002
d = 1536
faiss_index_m2 = faiss.IndexFlatL2(d)

vector_store_m2 = FaissVectorStore(faiss_index=faiss_index_m2)
storage_context_m2 = StorageContext.from_defaults(vector_store=vector_store_m2)
q1_2023_index_m = VectorStoreIndex.from_documents(q1_2023_new, storage_context=storage_context_m2)


In [ ]:
d = 1536
faiss_index_m3 = faiss.IndexFlatL2(d)

vector_store_m3 = FaissVectorStore(faiss_index=faiss_index_m3)
storage_context_m3 = StorageContext.from_defaults(vector_store=vector_store_m3)

q1_2024_index_m = VectorStoreIndex.from_documents(q1_2024_new, storage_context=storage_context_m3)

In [ ]:
q1_2022__engine_m = q1_2022_index_m.as_query_engine(similarity_top_k=2)
q1_2023__engine_m = q1_2023_index_m.as_query_engine(similarity_top_k=2)
q1_2024__engine_m = q1_2024_index_m.as_query_engine(similarity_top_k=2)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=q1_2022__engine_m,
        metadata=ToolMetadata(
            name="q1_2022",
            description="Provides quarterly performance report for infosys for first quarter of 2022",
        ),
    ),
    QueryEngineTool(
        query_engine=q1_2023__engine_m,
        metadata=ToolMetadata(
            name="q1_2023",
            description="Provides quarterly performance report for infosys for first quarter of 2023",
        ),
    ),
    QueryEngineTool(
        query_engine=q1_2024__engine_m,
        metadata=ToolMetadata(
            name="q1_2024",
            description="Provides quarterly performance report for infosys for first quarter of 2024",
        ),
    ),
]

In [ ]:
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [ ]:
response = s_engine.query(
    "What was the revenue growth in the first quarters of 2022, 2023 and 2024?"
)

Generated 3 sub questions.
[q1_2022] Q: What was the revenue growth in the first quarter of 2022?
[q1_2023] Q: What was the revenue growth in the first quarter of 2023?
[q1_2024] Q: What was the revenue growth in the first quarter of 2024?
[q1_2022] A: The revenue growth in the first quarter of 2022 was 16.9% year on year and 4.8% quarter on quarter in constant currency.
[q1_2023] A: The revenue growth in the first quarter of 2023 was 21.4% year-on-year and 5.5% sequentially in constant currency terms.
[q1_2024] A: The revenue growth in the first quarter of 2024 was 4.2%.


In [ ]:
print(response)

The revenue growth in the first quarters of 2022, 2023, and 2024 was 16.9%, 21.4%, and 4.2% respectively.


In [ ]:
response = s_engine.query(
    "How the total assets have changed from 2022 to 2024 year by year?"
)

Generated 3 sub questions.
[q1_2022] Q: What are the total assets for Infosys in Q1 2022?
[q1_2023] Q: What are the total assets for Infosys in Q1 2023?
[q1_2024] Q: What are the total assets for Infosys in Q1 2024?
[q1_2023] A: The total assets for Infosys in Q1 2023 are ₹119,979 crore.
[q1_2024] A: The total assets for Infosys in Q1 2024 are 131,322 crore.
[q1_2022] A: The total assets for Infosys in Q1 2022 are ₹109,486 crore.


In [ ]:
print(response)

The total assets for Infosys have increased from ₹109,486 crore in Q1 2022 to ₹119,979 crore in Q1 2023, and further increased to ₹131,322 crore in Q1 2024.


In [ ]:
response = s_engine.query(
    "Which are the key stragegic client wins or collborations that were achieved in 2022, 2023 and 2024?")

Generated 3 sub questions.
[q1_2022] Q: What were the key strategic client wins or collaborations achieved in 2022?
[q1_2023] Q: What were the key strategic client wins or collaborations achieved in 2023?
[q1_2024] Q: What were the key strategic client wins or collaborations achieved in 2024?
[q1_2024] A: Danske Bank, bp, Aramco, British Telecom, Walmart Commerce Technologies, and vidaXL were some of the key strategic client wins or collaborations achieved in 2024.
[q1_2023] A: The key strategic client wins or collaborations achieved in 2023 are not mentioned in the given context information.
[q1_2022] A: The key strategic client wins or collaborations achieved in 2022 include Archrock selecting Infosys for integrating digital technologies and mobile tools for its field service technicians, and Britvic selecting Infosys to help deliver their strategic transformation roadmap and operations. Additionally, Infosys collaborated with the French Tennis Federation to unveil new technologies t

In [ ]:
print(response)

The key strategic client wins or collaborations achieved in 2022 include Archrock selecting Infosys for integrating digital technologies and mobile tools for its field service technicians, and Britvic selecting Infosys to help deliver their strategic transformation roadmap and operations. Additionally, Infosys collaborated with the French Tennis Federation to unveil new technologies to enhance the Roland-Garros experience for the tennis ecosystem.

The key strategic client wins or collaborations achieved in 2023 are not mentioned in the given context information.

In 2024, some of the key strategic client wins or collaborations were with Danske Bank, bp, Aramco, British Telecom, Walmart Commerce Technologies, and vidaXL.


In [ ]:
response = s_engine.query(
    "Which was the message by the CEO in the press release of first quarter for the year 2024?")

Generated 1 sub questions.
[q1_2024] Q: What is the message by the CEO in the press release of first quarter for the year 2024?
[q1_2024] A: The CEO's message in the press release for the first quarter of 2024 is that the company had a solid Q1 with a growth of 4.2% and large deals of $2.3 billion, which helps to set a strong foundation for future growth. The CEO also mentioned that their generative AI capabilities are expanding well, with 80 active client projects, and that their comprehensive AI offering, Topaz, is resonating well with clients. The CEO sees this AI expansion as transformative for clients and enhancing their overall service portfolio. Additionally, the CEO mentioned that the company has expanded the margin improvement program with a holistic set of actions for the short, medium, and long-term, supported by their leadership team.


In [ ]:
print(response)

The CEO's message in the press release for the first quarter of 2024 highlighted the company's solid performance, with a growth of 4.2% and large deals of $2.3 billion. The CEO also emphasized the expansion of their generative AI capabilities, with 80 active client projects, and the positive reception of their comprehensive AI offering, Topaz. The CEO sees this AI expansion as transformative for clients and enhancing their overall service portfolio. Additionally, the CEO mentioned the company's expanded margin improvement program, which includes a holistic set of actions for the short, medium, and long-term, supported by their leadership team.
